# WEEK 3 TASK - TORONTO - E. Alvear

Import libraries

In [273]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Part I

Import the table from wikipedia

In [75]:
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
tables = pd.read_html(link)

This code is to take only the Postal Codes with a Borough assigned

In [76]:
Toronto_Data=tables[0]
Toronto_Data=Toronto_Data[Toronto_Data["Borough"]!="Not assigned"]
Toronto_Data

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
...,...,...,...
160,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,Business reply mail Processing Centre
169,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...


Here we replace the "/" with a "," in the boroughs with more than 2 Neighborhood

In [77]:
Toronto_Data["Neighborhood"]=Toronto_Data["Neighborhood"].str.replace('/.', ',', regex=True)
Toronto_Data

E:\Users\Erick\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park ,Harbourfront"
5,M6A,North York,"Lawrence Manor ,Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park ,Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway ,Montgomery Road ,Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,Business reply mail Processing Centre
169,M8Y,Etobicoke,"Old Mill South ,King's Mill Park ,Sunnylea ,Hu..."


Comproving if there is any "not assigned" Neighborhood (it looks that there isn't)

In [78]:
Toronto_Data[Toronto_Data["Neighborhood"]=="Not assigned"]

,Postal code,Borough,Neighborhood


In [79]:
Toronto_Data.shape

(103, 3)

## END OF THE PART I -----------------------------------------------------------------

### Part II

Importing the coordinates from the coursera URL

In [80]:
Coordinates=pd.read_csv("http://cocl.us/Geospatial_data")
Coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [81]:
Coordinates.shape

(103, 3)

With this line of code we merge the two DataFrames that have one column with the sames values, in this case "Postal Code"

In [82]:
Toronto_Data=Toronto_Data.merge(Coordinates,left_on='Postal code', right_on='Postal Code').drop(columns="Postal Code")
Toronto_Data.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park ,Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor ,Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park ,Ontario Provincial Government",43.662301,-79.389494


In [188]:
Toronto_Data.shape

(103, 5)

## END OF PART II ------------------------------------------------------

### PART III

If we explore the data we can see that tere are three Neigborhood that repeats

In [235]:
Toronto_Data.groupby("Neighborhood").count().sort_values(by=["Postal code"])

,Postal code,Borough,Latitude,Longitude
Neighborhood,,,,
Agincourt,1,1,1,1
"Runnymede ,Swansea",1,1,1,1
"Rouge Hill ,Port Union ,Highland Creek",1,1,1,1
Roselawn,1,1,1,1
Rosedale,1,1,1,1
...,...,...,...,...
"Kennedy Park ,Ionview ,East Birchmount Park",1,1,1,1
"Kensington Market ,Chinatown ,Grange Park",1,1,1,1
Don Mills,2,2,2,2


So lets only work only with one of each them, first let's find which are the index for this neigborhoods

In [239]:
print(Toronto_Data_A[Toronto_Data["Neighborhood"]=="Don Mills"])
print(Toronto_Data_A[Toronto_Data["Neighborhood"]=="Willowdale"])
print(Toronto_Data_A[Toronto_Data["Neighborhood"]=="Downsview"])

   Postal code     Borough Neighborhood   Latitude  Longitude
7          M3B  North York    Don Mills  43.745906 -79.352188
13         M3C  North York    Don Mills  43.725900 -79.340923
   Postal code     Borough Neighborhood   Latitude  Longitude
59         M2N  North York   Willowdale  43.770120 -79.408493
72         M2R  North York   Willowdale  43.782736 -79.442259
   Postal code     Borough Neighborhood   Latitude  Longitude
40         M3K  North York    Downsview  43.737473 -79.464763
46         M3L  North York    Downsview  43.739015 -79.506944
53         M3M  North York    Downsview  43.728496 -79.495697
60         M3N  North York    Downsview  43.761631 -79.520999


In [ ]:
Toronto_Data.drop([13,72,46,53,60])

We get the coordinates of Toronto in the Ontario State

In [250]:
address = 'TORONTO, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


We Draw the map and set marks in the neighbourhoods

In [251]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_Data['Latitude'], Toronto_Data['Longitude'], Toronto_Data['Borough'], Toronto_Data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Forsquare Credentials

In [252]:
CLIENT_ID = 'HFSRRHRGQKWKE45BO1AD0HNV3V3H0BG4FHRYERH5HDY5LVO4' 
CLIENT_SECRET = 'IMIGNU3BPFMXYQEKX2LTIRO3CSA43L5ZVDJJHLJE5DQ21R5P' 
VERSION = '20180605' 
LIMIT=100



We define the function that gets 100 venues near every neighborhood

In [253]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Then we run the function for every one of the neigbourhoods in Toronto

In [254]:
Toronto_venues = getNearbyVenues(names=Toronto_Data['Neighborhood'],
                                   latitudes=Toronto_Data['Latitude'],
                                   longitudes=Toronto_Data['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park ,Harbourfront
Lawrence Manor ,Lawrence Heights
Queen's Park ,Ontario Provincial Government
Islington Avenue
Malvern ,Rouge
Don Mills
Parkview Hill ,Woodbine Gardens
Garden District ,Ryerson
Glencairn
West Deane Park ,Princess Gardens ,Martin Grove ,Islington ,Cloverdale
Rouge Hill ,Port Union ,Highland Creek
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate ,Bloordale Gardens ,Old Burnhamthorpe ,Markland Wood
Guildwood ,Morningside ,West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor ,Wilson Heights ,Downsview North
Thorncliffe Park
Richmond ,Adelaide ,King
Dufferin ,Dovercourt Village
Scarborough Village
Fairview ,Henry Farm ,Oriole
Northwood Park ,York University
East Toronto
Harbourfront East ,Union Station ,Toronto Islands
Little Portugal ,Trinity
Kennedy Park ,Ionview ,East Birchmount Park
Bayview Village
Downsview
The Danforth West ,Riverdale
To

In [255]:
Toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [342]:
Toronto_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood ,Long Branch",10,10,10,10,10,10
"Bathurst Manor ,Wilson Heights ,Downsview North",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park ,Lawrence Manor East",23,23,23,23,23,23


Now let's see which are the most common venues for each neighbourhood

In [257]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [258]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1,"Alderwood ,Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor ,Wilson Heights ,Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.050000,0.000000,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park ,Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,Willowdale,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.029412,0.0,0.0,0.0,0.0,0.0
89,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
90,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.083333,0.000000,0.0,0.0,0.0,0.0,0.0
91,"York Mills ,Silver Hills",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


This Function will help us to get the most common venues of each Neighbourood

In [301]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now we create a DataFrame with the top 10 venues for each Neighbourhood

In [359]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Clothing Store,Breakfast Spot,Doner Restaurant,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run
1,"Alderwood ,Long Branch",Pizza Place,Sandwich Place,Pool,Gym,Coffee Shop,Pub,Skating Rink,Pharmacy,Athletics & Sports,Deli / Bodega
2,"Bathurst Manor ,Wilson Heights ,Downsview North",Coffee Shop,Bank,Middle Eastern Restaurant,Pharmacy,Sushi Restaurant,Fried Chicken Joint,Restaurant,Diner,Pizza Place,Bridal Shop
3,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Women's Store,Department Store,Diner,Discount Store,Distribution Center,Dog Run
4,"Bedford Park ,Lawrence Manor East",Sandwich Place,Coffee Shop,Italian Restaurant,Restaurant,Liquor Store,Pharmacy,Butcher,Café,Sushi Restaurant,Hobby Shop


Now let's make some clusters

In [360]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

kmeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 4, 1, 0, 1, 1, 1, 4, 1, 4, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1,
       1, 1, 4, 1, 1, 1, 0, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 0])

Now let's add the clusters to the DataFrame

In [361]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_Data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head(6)# check the last columns!


,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,4.0,Park,Food & Drink Shop,Women's Store,Dog Run,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Doner Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Portuguese Restaurant,Hockey Arena,Intersection,French Restaurant,Coffee Shop,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
2,M5A,Downtown Toronto,"Regent Park ,Harbourfront",43.654260,-79.360636,1.0,Coffee Shop,Pub,Bakery,Park,Café,Theater,Breakfast Spot,Health Food Store,Historic Site,Hotel
3,M6A,North York,"Lawrence Manor ,Lawrence Heights",43.718518,-79.464763,1.0,Furniture / Home Store,Clothing Store,Women's Store,Vietnamese Restaurant,Coffee Shop,Boutique,Gift Shop,Accessories Store,Miscellaneous Shop,Event Space
4,M7A,Downtown Toronto,"Queen's Park ,Ontario Provincial Government",43.662301,-79.389494,1.0,Coffee Shop,Sushi Restaurant,Diner,Mexican Restaurant,Bar,Beer Bar,Sandwich Place,Burrito Place,Café,College Auditorium
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


As we can see, there are some Neighborhoods without cluster, this is because foursquare cant found anything around them, so let's drop them

In [362]:
Toronto_merged.dropna(subset=["Cluster Labels"],axis=0,inplace=True)
Toronto_merged["Cluster Labels"]=Toronto_merged["Cluster Labels"].astype(int)
Toronto_merged.head(6)

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,4,Park,Food & Drink Shop,Women's Store,Dog Run,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Doner Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,1,Portuguese Restaurant,Hockey Arena,Intersection,French Restaurant,Coffee Shop,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
2,M5A,Downtown Toronto,"Regent Park ,Harbourfront",43.654260,-79.360636,1,Coffee Shop,Pub,Bakery,Park,Café,Theater,Breakfast Spot,Health Food Store,Historic Site,Hotel
3,M6A,North York,"Lawrence Manor ,Lawrence Heights",43.718518,-79.464763,1,Furniture / Home Store,Clothing Store,Women's Store,Vietnamese Restaurant,Coffee Shop,Boutique,Gift Shop,Accessories Store,Miscellaneous Shop,Event Space
4,M7A,Downtown Toronto,"Queen's Park ,Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Diner,Mexican Restaurant,Bar,Beer Bar,Sandwich Place,Burrito Place,Café,College Auditorium
6,M1B,Scarborough,"Malvern ,Rouge",43.806686,-79.194353,1,Fast Food Restaurant,Print Shop,Women's Store,Dog Run,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Doner Restaurant


In [363]:
Toronto_merged.groupby("Cluster Labels").count().sort_values(by=["Postal code"])

,Postal code,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Cluster Labels,,,,,,,,,,,,,,,
2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
0,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
4,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8
1,78,78,78,78,78,78,78,78,78,78,78,78,78,78,78


In [364]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels'].astype(int)):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

###  1) It seems that most of the neighbourhoods in Toronto are the same (group 1).

### 2)  It seems that the York Mills neighborhood is unique because it is the only one in group 2, let's see what the most common places are there.

In [366]:
Toronto_merged[Toronto_merged["Cluster Labels"]==2]

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,M2L,North York,"York Mills ,Silver Hills",43.75749,-79.374714,2,Cafeteria,Dance Studio,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run


# Thank You

### E. Alvear